In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf
import pathlib

TRAIN_DATA_URL = pathlib.Path('/content/drive/MyDrive/1678463496train.csv')
TEST_DATA_URL = pathlib.Path('/content/drive/MyDrive/1678463949eval.csv')
 
train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

# Haremos valores de Numpy faciles de leer
np.set_printoptions(precision=3, suppress=True)

!head {train_file_path}

LABEL_COLUMN = 'survived'
LABELS = [0, 1]

def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Un tamaño de 5 para que sea más fácil de mostrar
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

show_batch(raw_train_data)

CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

example_batch, labels_batch = next(iter(temp_dataset)) 

def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

packed_dataset = temp_dataset.map(pack)


for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

show_batch(raw_train_data)

example_batch, labels_batch = next(iter(temp_dataset)) 

class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_freatures = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_freatures]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(PackNumericFeatures(NUMERIC_FEATURES))

show_batch(packed_train_data)

example_batch, labels_batch = next(iter(packed_train_data))

import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

def normalize_numeric_data(data, mean, std):
  # Centramos la data
  return (data-mean)/std


# Se verá lo que se ha creado
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

example_batch['numeric']

numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

  # Se verá lo que se ha creado
categorical_columns

categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

print(preprocessing_layer(example_batch).numpy()[0])

model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

model.fit(train_data, epochs=20)

test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

predictions = model.predict(test_data)

# Veremos los resultados de la prediccion
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n
sex                 : [b'female' b'male' b'male' b'male' b'male']
age                 : [17. 28. 28. 28. 31.]
n_siblings_spouses  : [1 0 0 0 1]
parch               : [0 0 0 0 1]
fare                : [57.     6.95  26.     7.75  37.004]
class               : [b'First' b'Third' b'First' b'Third' b'Second']
deck                : [b'B' b'unknown' b'A' b'unknown' b'unknown']
embark_town         : [b'Southampton

126/126 [==============================] - 2s 3ms/step - loss: 0.5124 - accuracy: 0.7544
Epoch 2/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4261 - accuracy: 0.8214
Epoch 3/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4033 - accuracy: 0.8262
Epoch 4/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3966 - accuracy: 0.8405
Epoch 5/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3852 - accuracy: 0.8325
Epoch 6/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3859 - accuracy: 0.8325
Epoch 7/20
126/126 [==============================] - 1s 4ms/step - loss: 0.3725 - accuracy: 0.8453
Epoch 8/20
126/126 [==============================] - 1s 4ms/step - loss: 0.3628 - accuracy: 0.8469
Epoch 9/20
126/126 [==============================] - 1s 3ms/step - loss: 0.3618 - accuracy: 0.8405
Epoch 10/20
126/126 [==============================] - 1s 4ms/step - loss: 0.3500 - accuracy: 0.8517
Epoch 11/2

53/53 [==============================] - 0s 2ms/step - loss: 0.4772 - accuracy: 0.8068




Test Loss 0.4772421717643738, Test Accuracy 0.8068181872367859
53/53 [==============================] - 0s 2ms/step
Predicted survival: 48.55%  | Actual outcome:  SURVIVED
Predicted survival: 77.49%  | Actual outcome:  DIED
Predicted survival: 49.27%  | Actual outcome:  SURVIVED
Predicted survival: 99.99%  | Actual outcome:  DIED
Predicted survival: 15.34%  | Actual outcome:  DIED
